<a href="https://colab.research.google.com/github/ketanp23/sit-neuralnetworks-class/blob/main/Function_Approximation_and_Generalization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt

# --- 1. Define the True Underlying Function ---
def target_function(x):
    """The function we are trying to approximate: f(x) = 0.5 * sin(2*pi*x) + x"""
    return 0.5 * np.sin(2 * np.pi * x) + x

# --- 2. Data Generation ---
N_SAMPLES = 100
NOISE_STD = 0.1 # Standard deviation of Gaussian noise

# Generate 100 random input points between 0 and 1
X_train_np = np.sort(np.random.rand(N_SAMPLES, 1), axis=0)

# Calculate true y values and add noise
Y_true_np = target_function(X_train_np)
Y_train_np = Y_true_np + np.random.normal(0, NOISE_STD, Y_true_np.shape)

# Convert NumPy arrays to PyTorch Tensors
X_train = torch.tensor(X_train_np, dtype=torch.float32)
Y_train = torch.tensor(Y_train_np, dtype=torch.float32)

print(f"Generated {N_SAMPLES} noisy training samples.")

# --- 3. Define the Neural Network Model ---
class FunctionApproximator(nn.Module):
    def __init__(self):
        super().__init__()
        # Input layer (1 feature: x) -> Hidden layer 1
        self.fc1 = nn.Linear(1, 50)
        # Hidden layer 1 -> Hidden layer 2
        self.fc2 = nn.Linear(50, 50)
        # Hidden layer 2 -> Output layer (1 output: y)
        self.fc3 = nn.Linear(50, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Initialize the model, loss, and optimizer
model = FunctionApproximator()
criterion = nn.MSELoss() # Mean Squared Error is standard for regression/approximation
optimizer = optim.Adam(model.parameters(), lr=0.001)

# --- 4. Training Loop ---
NUM_EPOCHS = 5000
print(f"Starting training for {NUM_EPOCHS} epochs...")

for epoch in range(NUM_EPOCHS):
    # Forward pass
    outputs = model(X_train)
    loss = criterion(outputs, Y_train)

    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 1000 == 0:
        print(f'Epoch [{epoch+1}/{NUM_EPOCHS}], Loss: {loss.item():.6f}')

print("Training finished.")

# --- 5. Test for Generalization and Visualization ---

# Generate clean, dense data points for testing generalization (unseen data)
# This allows us to see the smoothness of the learned function vs. the true function.
X_test_np = np.linspace(0, 1, 500).reshape(-1, 1)
Y_true_clean_np = target_function(X_test_np)
X_test = torch.tensor(X_test_np, dtype=torch.float32)

# Make predictions using the trained model
model.eval() # Set model to evaluation mode
with torch.no_grad():
    Y_pred = model(X_test).numpy()

# Calculate the error on the clean test set (generalization error)
generalization_loss = np.mean((Y_pred - Y_true_clean_np)**2)

# --- 6. Plotting the Results ---
plt.figure(figsize=(10, 6))

# Plot 1: The True Function (for comparison)
plt.plot(X_test_np, Y_true_clean_np, label='True Function: $f(x)$', color='blue', linewidth=3, linestyle='--')

# Plot 2: The Noisy Training Data (what the model saw)
plt.scatter(X_train_np, Y_train_np, label='Noisy Training Data', color='red', alpha=0.6, s=20)

# Plot 3: The Learned/Approximated Function (The generalization result)
plt.plot(X_test_np, Y_pred, label='Learned Function (Approximation)', color='green', linewidth=3)

plt.title(f'Function Approximation and Generalization (MSE on Test: {generalization_loss:.4e})', fontsize=14)
plt.xlabel('Input X', fontsize=12)
plt.ylabel('Output Y', fontsize=12)
plt.legend()
plt.grid(True, linestyle='--')
plt.savefig('function_approximation_result.png')
plt.close()

# --- 7. Print Final Output ---
print("-" * 50)
print(f"Final Training Loss: {loss.item():.6f}")
print(f"Generalization MSE on Clean Test Data: {generalization_loss:.4e}")
print("\nVisualization saved to 'function_approximation_result.png'")
print("The plot demonstrates generalization: the green line (Learned Function) fits the blue line (True Function) well, ignoring the scattered red dots (Noise).")

Generated 100 noisy training samples.
Starting training for 5000 epochs...
Epoch [1000/5000], Loss: 0.007523
Epoch [2000/5000], Loss: 0.007335
Epoch [3000/5000], Loss: 0.007192
Epoch [4000/5000], Loss: 0.007105
Epoch [5000/5000], Loss: 0.007041
Training finished.
--------------------------------------------------
Final Training Loss: 0.007041
Generalization MSE on Clean Test Data: 1.6891e-03

Visualization saved to 'function_approximation_result.png'
The plot demonstrates generalization: the green line (Learned Function) fits the blue line (True Function) well, ignoring the scattered red dots (Noise).
